In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [2]:
# Path to the data txt file on disk.
data_path = "D://ukr.txt"
# open the file eng-spa.txt and read
lines= pd.read_table(data_path,  names =['source', 'target', 'comments'])
#printing sample data from lines
lines.sample(6)

,source,target,comments
75868,Has Tom already been here?,Том вже був тут?,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
112222,I doubt that Tom has to do that.,"Маю сумнів, що Том має це робити.",CC-BY 2.0 (France) Attribution: tatoeba.org #7...
112832,I want to take your temperature.,Я хочу поміряти твою температуру.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1444,Come to me.,Йди до мене.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
30577,It could be a trap.,Це може бути пасткою.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
106862,I forgot to close the curtains.,Я забув закрити штори.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [3]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "D://ukr.txt"

In [4]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 10000
Number of unique input tokens: 72
Number of unique output tokens: 95
Max sequence length for inputs: 15
Max sequence length for outputs: 54


In [5]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [6]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")

Epoch 1/100
125/125 [==============================] - 31s 224ms/step - loss: 1.1211 - accuracy: 0.7453 - val_loss: 0.9878 - val_accuracy: 0.7337
Epoch 2/100
125/125 [==============================] - 27s 217ms/step - loss: 0.8109 - accuracy: 0.7885 - val_loss: 0.7894 - val_accuracy: 0.7865
Epoch 3/100
125/125 [==============================] - 27s 214ms/step - loss: 0.6853 - accuracy: 0.8123 - val_loss: 0.6898 - val_accuracy: 0.8080
Epoch 4/100
125/125 [==============================] - 26s 210ms/step - loss: 0.5914 - accuracy: 0.8313 - val_loss: 0.6340 - val_accuracy: 0.8177
Epoch 5/100
125/125 [==============================] - 27s 213ms/step - loss: 0.5479 - accuracy: 0.8408 - val_loss: 0.6132 - val_accuracy: 0.8219
Epoch 6/100
125/125 [==============================] - 27s 213ms/step - loss: 0.5166 - accuracy: 0.8484 - val_loss: 0.5720 - val_accuracy: 0.8318
Epoch 7/100
125/125 [==============================] - 27s 213ms/step - loss: 0.4898 - accuracy: 0.8558 - val_loss: 0.5584 -

125/125 [==============================] - 32s 255ms/step - loss: 0.0972 - accuracy: 0.9700 - val_loss: 0.6214 - val_accuracy: 0.8689
Epoch 57/100
125/125 [==============================] - 30s 243ms/step - loss: 0.0948 - accuracy: 0.9706 - val_loss: 0.6262 - val_accuracy: 0.8687
Epoch 58/100
125/125 [==============================] - 31s 251ms/step - loss: 0.0926 - accuracy: 0.9715 - val_loss: 0.6331 - val_accuracy: 0.8689
Epoch 59/100
125/125 [==============================] - 32s 254ms/step - loss: 0.0901 - accuracy: 0.9721 - val_loss: 0.6392 - val_accuracy: 0.8690
Epoch 60/100
125/125 [==============================] - 32s 254ms/step - loss: 0.0880 - accuracy: 0.9726 - val_loss: 0.6430 - val_accuracy: 0.8690
Epoch 61/100
125/125 [==============================] - 31s 252ms/step - loss: 0.0860 - accuracy: 0.9731 - val_loss: 0.6492 - val_accuracy: 0.8689
Epoch 62/100
125/125 [==============================] - 32s 254ms/step - loss: 0.0838 - accuracy: 0.9739 - val_loss: 0.6514 - val_a

INFO:tensorflow:Assets written to: s2s\assets


INFO:tensorflow:Assets written to: s2s\assets


In [12]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [13]:
for seq_index in range(0,1000,25):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

Input sentence: Go.
Decoded sentence: Йди.

Input sentence: Do it.
Decoded sentence: Дійте.

Input sentence: Eat it.
Decoded sentence: З'їж його.

Input sentence: I paid.
Decoded sentence: Я заплатила.

Input sentence: Thanks!
Decoded sentence: Дякую!

Input sentence: Be fair.
Decoded sentence: Будьте справедливі.

Input sentence: Get out.
Decoded sentence: Йди звідси.

Input sentence: Help me!
Decoded sentence: Допоможіть мені!

Input sentence: I dozed.
Decoded sentence: Я задрімав.

Input sentence: I'm mad.
Decoded sentence: Я розлючений.

Input sentence: Lock it.
Decoded sentence: Замкніть його.

Input sentence: Wash up.
Decoded sentence: Умийся.

Input sentence: You won.
Decoded sentence: Ви виграли.

Input sentence: Catch me.
Decoded sentence: Злови мене.

Input sentence: Get lost!
Decoded sentence: Йди геть.

Input sentence: Humor me.
Decoded sentence: Зробіть це для мене.

Input sentence: I forgot.
Decoded sentence: Я забув.

Input sentence: I paused.
Decoded sentence: Я зробив 